https://towardsdatascience.com/keras-data-generators-and-how-to-use-them-b69129ed779c

https://medium.com/@vijayabhaskar96/tutorial-on-keras-flow-from-dataframe-1fd4493d237c

In [14]:
!pip install tqdm
!pip install tqdm --upgrade

     |████████████████████████████████| 61kB 8.2MB/s 
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1


In [0]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import PIL
import os
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input,Conv2D,MaxPooling2D,UpSampling2D,ZeroPadding2D,Dense,Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
import pandas as pd
import numpy as np
from tqdm.keras import TqdmCallback

In [2]:
!unzip '/content/drive/My Drive/Máster KSchool/TFM/data/filtered_img.zip'

Streaming output truncated to the last 5000 lines.
  inflating: filtered_img/tt0246262.png  
  inflating: filtered_img/tt0269281.png  
  inflating: filtered_img/tt0011396.png  
  inflating: filtered_img/tt0078320.png  
  inflating: filtered_img/tt0389361.png  
  inflating: filtered_img/tt0092921.png  
  inflating: filtered_img/tt0115682.png  
  inflating: filtered_img/tt0029609.png  
  inflating: filtered_img/tt0123330.png  
  inflating: filtered_img/tt2006093.png  
  inflating: filtered_img/tt0061671.png  
  inflating: filtered_img/tt0953903.png  
  inflating: filtered_img/tt0322082.png  
  inflating: filtered_img/tt0028517.png  
  inflating: filtered_img/tt4799672.png  
  inflating: filtered_img/tt1305714.png  
  inflating: filtered_img/tt0284137.png  
  inflating: filtered_img/tt0201129.png  
  inflating: filtered_img/tt0038704.png  
  inflating: filtered_img/tt3163244.png  
  inflating: filtered_img/tt0227850.png  
  inflating: filtered_img/tt4799114.png  
  inflating: filtered_img

In [0]:
data_tab = pd.read_csv('/content/drive/My Drive/Máster KSchool/TFM/data/filtered_df_with_imgNames.csv',sep=',',header= 0,usecols=['CODE','RATING'])
data_tab = data_tab.apply(lambda x: x.str.replace(',','.'))

In [4]:
data_tab.head(3)

,CODE,RATING
0,tt4686108.png,5.3
1,tt0445033.png,6.4
2,tt1167599.png,5.5


Categorizamos la columna Rating


In [5]:
data_tab['RATING']  = data_tab['RATING'].astype(float)
data_tab['RATING'] = pd.cut( data_tab['RATING'], bins = [0,5, 6.5,10], labels=['1','2','3'])
data_tab

,CODE,RATING
0,tt4686108.png,2
1,tt0445033.png,2
2,tt1167599.png,2
3,tt4823434.png,3
4,tt0220100.png,2
...,...,...
107176,tt0099876.png,3
107177,tt0052564.png,2
107178,tt0051601.png,2
107179,tt0151309.png,2


In [6]:
print(len(data_tab[data_tab['RATING']==3]))
print(len(data_tab[data_tab['RATING']==2]))
print(len(data_tab[data_tab['RATING']==1]))

0
0
0


Leemos la IMG

In [7]:
'''img_list_names = os.listdir('filtered_img/')
img_list_names.remove('.DS_Store')

array_img = pd.DataFrame(columns = ['CODE','IMG_VALUES'])
for i,id_image in enumerate(tqdm(img_list_names)):
        img = PIL.Image.open('filtered_img/' + id_image)
        img_array = np.array(img)
        array_img.loc[i] = [id_image]+[img_array] '''

"img_list_names = os.listdir('filtered_img/')\nimg_list_names.remove('.DS_Store')\n\narray_img = pd.DataFrame(columns = ['CODE','IMG_VALUES'])\nfor i,id_image in enumerate(tqdm(img_list_names)):\n        img = PIL.Image.open('filtered_img/' + id_image)\n        img_array = np.array(img)\n        array_img.loc[i] = [id_image]+[img_array] "

Image data generator

In [8]:
datagen=ImageDataGenerator(rescale=1./255.,validation_split=0.25)

train_generator=datagen.flow_from_dataframe(
                        dataframe=data_tab,
                        directory="filtered_img/",
                        x_col="CODE",
                        y_col="RATING",
                        subset="training",
                        batch_size=32,
                        seed=42,
                        shuffle=True,
                        class_mode="categorical",
                        target_size=(268, 182))
valid_generator=datagen.flow_from_dataframe(
                        dataframe=data_tab,
                        directory="filtered_img/",
                        x_col="CODE",
                        y_col="RATING",
                        subset="validation",
                        batch_size=32,
                        seed=42,
                        shuffle=True,
                        class_mode="categorical",
                        target_size=(268,182))

Found 80386 validated image filenames belonging to 3 classes.
Found 26795 validated image filenames belonging to 3 classes.


In [9]:
model = Sequential()
model.add(tf.keras.layers.Conv2D(32, (3, 3), padding='same',input_shape=(268,182,3)))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Conv2D(32, (3, 3)))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Conv2D(64, (3, 3), padding='same'))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Conv2D(64, (3, 3)))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(512))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(3, activation='softmax'))
model.compile(tf.keras.optimizers.RMSprop(lr=0.0001, decay=1e-6),loss="categorical_crossentropy",metrics=["accuracy"])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size

model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10,callbacks=[TqdmCallback()], verbose=2
)

Epoch 1/10
